### Spark Aggregate

#### Funciones simples de agregación

In [0]:
from pyspark.sql.functions import count,count_distinct,sum,max,min,avg

In [0]:
%run configuration

In [0]:
df_movies = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
display(df_movies)

In [0]:
df_movies.select(count("*")).show()

In [0]:
df_movies.select(count_distinct("year_release_date")).show()

In [0]:
display(df_movies.filter(df_movies.year_release_date==2016).select(sum(df_movies.budget),count("*")) \
        .withColumnRenamed("sum(budget)","total_budget").withColumnRenamed("count(1)","total_movies")
     )  

#### Group By

In [0]:
display(df_movies.groupBy(df_movies.year_release_date) \
                 .agg(
                    sum("budget").alias("total"),
                    max("budget").alias("max"),
                    min("budget").alias("min"),
                    count("movie_id").alias("total"),
                    avg("budget").alias("promedio")
                 )
                 )

#### Window functions

In [0]:
from pyspark.sql.functions import rank,desc,isnotnull
from pyspark.sql.window import Window

display(df_movies.filter(isnotnull(df_movies.year_release_date)).select(df_movies.title,df_movies.budget,df_movies.year_release_date) \
        .withColumn("rank",rank().over(Window.partitionBy("year_release_date").orderBy(desc("budget"))))
)

In [0]:
from pyspark.sql.functions import rank,desc,isnotnull,dense_rank
from pyspark.sql.window import Window

movie_dense_rank = Window.partitionBy("year_release_date").orderBy(desc("budget"))

display(df_movies.filter(isnotnull(df_movies.year_release_date)).select(df_movies.title,df_movies.budget,df_movies.year_release_date) \
        .withColumn("rank",dense_rank().over(movie_dense_rank))
)